In [ ]:
import numpy as np
import cvxpy as cp

from problems.data_generation import generate_problem_data
from problems.problem_definition import BilevelProblem
from algorithms.barrier_blo_box import BarrierBLO
from algorithms.blocc import BLOCC
from algorithms.implicit_gradient_descent import IGD

n = 60
seed = 9

data = generate_problem_data(n, seed)
problem = BilevelProblem(data)

x_init = np.zeros(n)
y_g_init = np.zeros(n)
y_F_init = np.zeros(n)
mu_g_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
mu_F_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
hparams = {
        'barrier_blo': {
            'M': 0.001,
            't': 0.01,
            'alpha_x': 0.002,
            'alpha_y': 0.1,
            'beta_y': 1,
            'epsilon_x': 0.1,
            'epsilon_y': 0.01,
            'inner_max_iters': 1000000,
            'outer_max_iters': 100000
        },
        'blocc': {
            'alpha': 1e-3,
            'gamma': 10,
            'alpha_g_y': 0.000001,
            'alpha_F_y': 0.000001,
            'beta_g_y': 0.000001,
            'beta_F_y': 0.000001,
            'epsilon_x': 0.1,
            'epsilon_inner_y_g': 0.01,
            'epsilon_outer_y_g': 0.01,
            'epsilon_inner_y_F': 0.01,
            'epsilon_outer_y_F': 0.01,
            'maxmin_g_outer_max_iters': 5000,
            'maxmin_F_outer_max_iters': 5000,
            'maxmin_g_inner_max_iters': 5000,
            'maxmin_F_inner_max_iters': 5000,
            'main_max_iters': 5000 
        },
        'IGD': {
            'M': 1e-3,
            't': 1e-3,
            'alpha_x': 0.001,
            'alpha_y': 0.001,
            'epsilon_x': 1e-3,
            'epsilon_y': 1e-3,
            'inner_max_iters': 1000,
            'outer_max_iters': 1000
        }
    }

# BLOCC Algorithm

In [4]:
blocc_algo = BLOCC(problem, hparams)


x_opt_blocc, y_opt_blocc, history = blocc_algo.blocc(x_init, y_init, mu_g_init, mu_F_init)
print(problem.f(x_opt_blocc, y_opt_blocc))



Main iteration 1
Inner loop for L_g converges when iter = 0
Outer iter for L_g=0, Projected Gradient norm w.r.t mu of L_g=0.0
Inner loop for L_F converges when iter = 0
Outer iter for L_F=0, Projected Gradient norm w.r.t mu of L_F=0.0
f(x,y)=-5.939507248070063,grad_norm = 79.45604905569178
Main iteration 2
Inner loop for L_g converges when iter = 0
Outer iter for L_g=0, Projected Gradient norm w.r.t mu of L_g=0.0
Inner loop for L_F converges when iter = 0
Outer iter for L_F=0, Projected Gradient norm w.r.t mu of L_F=0.0
f(x,y)=-10.53822946916874,grad_norm = 69.90321412439353
Main iteration 3
Inner loop for L_g converges when iter = 0
Outer iter for L_g=0, Projected Gradient norm w.r.t mu of L_g=0.0
Inner loop for L_F converges when iter = 0
Outer iter for L_F=0, Projected Gradient norm w.r.t mu of L_F=0.0
f(x,y)=-14.099720835761065,grad_norm = 61.50382566614957
Main iteration 4
Inner loop for L_g converges when iter = 0
Outer iter for L_g=0, Projected Gradient norm w.r.t mu of L_g=0.0


# BLOCC Feasibility Check

In [5]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h2)]

y_original_opt_blocc = barrier_algo.Interior_inner_loop(x_opt_blocc, y_opt_blocc)
print(f"f_theoretical_opt_result={problem.f(x_opt_blocc, y_original_opt_blocc)}, f_blocc_numerical_result={problem.f(x_opt_blocc, y_opt_blocc)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_blocc, y_original_opt_blocc)}, g_blocc_numerical_result={problem.g(x_opt_blocc, y_opt_blocc)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=56.85491651405927, f_blocc_numerical_result=-26.565363523750484
g_theoretical_opt_result=-48.518704028990456, g_blocc_numerical_result=-3.694499272996327
h_1[0] = -0.2733790647514564
h_1[1] = -0.8505180374364314
h_1[2] = -0.4921977053044375
h_1[3] = -0.33888150286232804
h_1[4] = -0.29107761468257337
h_1[5] = -0.04909726715175735
h_1[6] = -0.860027305785324
h_1[7] = -0.6355613852209886
h_1[8] = -0.3353268619324695
h_1[9] = -0.941424203167046
h_1[10] = -0.1290561751789365
h_1[11] = -0.3376950388760816
h_1[12] = -0.343120242624369
h_1[13] = -0.8986264688933713
h_1[14] = -0.708583509322275
h_1[15] = -0.32785728497241157
h_1[16] = -0.42610711652684174
h_1[17] = -0.34431362246186775
h_1[18] = -0.8549154143280508
h_1[19] = -0.007561306014173326
h_2[0] = -1.0041953753187072
h_2[1] = -0.9979405341032653
h_2[2] = -1.0018947438661925
h_2[3] = -1.0010698896811014
h_2[4] = -1.0027494228257487
h_2[5] = -0.996553306960922
h_2[6] = -1.0021215766354359
h_2[7] = -1.0064038702147

# Implicit Gradient Descent Algorithm

In [49]:
IGD_algo = IGD(problem, hparams)

x_opt_IGD, y_opt_IGD, history = IGD_algo.upper_loop(x_init, y_init)

print("Optimized x:", x_opt_IGD)
print("Optimized y:", y_opt_IGD)

h1_values = [problem.h_1(x_opt_IGD, y_opt_IGD, i) for i in range(problem.num_constraints_h1)]

print("\nValues of h_1 constraints at the optimal point:")
for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

times = [h['time'] for h in history]
grad_norms = [h['grad_norm'] for h in history]

Outer iteration 1
Inner loop converged at iteration 200
f(x, y) = 95.45037851360806, grad_norm = 76.59673418115331
Outer iteration 2
Inner loop converged at iteration 112
f(x, y) = 91.19500851740221, grad_norm = 67.2257525452191
Outer iteration 3
Inner loop converged at iteration 110
f(x, y) = 87.9160198480594, grad_norm = 59.00630049516197
Outer iteration 4
Inner loop converged at iteration 107
f(x, y) = 85.39015297241798, grad_norm = 51.796437571852486
Outer iteration 5
Inner loop converged at iteration 105
f(x, y) = 83.44417540097366, grad_norm = 45.471557030758525
Outer iteration 6
Inner loop converged at iteration 102
f(x, y) = 81.94475905747844, grad_norm = 39.922553604489096
Outer iteration 7
Inner loop converged at iteration 100
f(x, y) = 80.7892951506114, grad_norm = 35.053784045327106
Outer iteration 8
Inner loop converged at iteration 98
f(x, y) = 79.8987827656407, grad_norm = 30.781487703596458
Outer iteration 9
Inner loop converged at iteration 95
f(x, y) = 79.212391899265

# Implicit Gradient Descent Feasibility Check

In [50]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_IGD, y_opt_IGD, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_IGD, y_opt_IGD, i) for i in range(problem.num_constraints_h2)]

y_original_opt_IGD = barrier_algo.Interior_inner_loop(x_opt_IGD, y_opt_IGD)
print(f"f_theoretical_opt_result={problem.f(x_opt_IGD, y_original_opt_IGD)}, f_IGD_numerical_result={problem.f(x_opt_IGD, y_opt_IGD)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_IGD, y_original_opt_IGD)}, g_IGD_numerical_result={problem.g(x_opt_IGD, y_opt_IGD)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=59.04726374415966, f_IGD_numerical_result=76.90380080947456
g_theoretical_opt_result=-47.905074255466786, g_IGD_numerical_result=-51.811093935401786
h_1[0] = -1.818606806017621
h_1[1] = -0.6332653285456722
h_1[2] = 0.692060467876725
h_1[3] = -0.19201660481909777
h_1[4] = -0.600298040133991
h_1[5] = -0.2737396315591377
h_1[6] = -1.012366935480417
h_1[7] = 0.4450296606186358
h_1[8] = -1.4287421110645298
h_1[9] = -0.8173446049192401
h_1[10] = 1.0874902560609827
h_1[11] = -1.4789022155643556
h_1[12] = 0.10827124470942368
h_1[13] = -1.786263928772887
h_1[14] = -0.8903993418731315
h_1[15] = -0.07504978497512865
h_1[16] = -0.3015547409733911
h_1[17] = -0.21462543837571568
h_1[18] = -0.693290361744195
h_1[19] = 0.5416734352752851
h_2[0] = -1.0929089554460576
h_2[1] = -0.9654870816489937
h_2[2] = -1.0237375559765334
h_2[3] = -1.0192583555543342
h_2[4] = -1.044948936664087
h_2[5] = -0.877464459734365
h_2[6] = -1.0860014094637798
h_2[7] = -1.2018995373962
h_2[8] = -1.4768

# BFBM Algorithm

In [48]:
barrier_algo = BarrierBLO(problem, hparams)

# y_proj = barrier_algo.project_to_constraints(x_init, y_init)

# h = problem.h_1(x_init, y_proj, 0)

# print(y_proj)

# print(h)



x_opt_barrier, y_opt_barrier, history = barrier_algo.upper_loop(x_init, y_init)

print("Optimized x:", x_opt_barrier)
print("Optimized y:", y_opt_barrier)

h1_values = [problem.h_1(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]

print("\nValues of h_1 constraints at the optimal point:")
for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

times = [h['time'] for h in history]
grad_norms = [h['grad_norm'] for h in history]

Outer iteration 1
Inner loop converged at iteration 86
f(x, y) = 90.49973731529963, grad_norm of hyperfunction= 77.86208771566766
Outer iteration 2
Inner loop converged at iteration 66
f(x, y) = 105.03488086901551, grad_norm of hyperfunction= 516.5132229121378
Outer iteration 3
Inner loop converged at iteration 95
f(x, y) = 39.34324259400703, grad_norm of hyperfunction= 154.56844579751348
Outer iteration 4
Inner loop converged at iteration 95
f(x, y) = 18.239834708638128, grad_norm of hyperfunction= 127.14698119973167
Outer iteration 5
Inner loop converged at iteration 92
f(x, y) = 6.596745179997072, grad_norm of hyperfunction= 95.77394264819782
Outer iteration 6
Inner loop converged at iteration 92
f(x, y) = 0.6448443854900461, grad_norm of hyperfunction= 75.49432363158589
Outer iteration 7
Inner loop converged at iteration 92
f(x, y) = -2.781235525332633, grad_norm of hyperfunction= 61.463932878767785
Outer iteration 8
Inner loop converged at iteration 92
f(x, y) = -4.937869237625001

In [46]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h2)]

y_original_opt_bfbm = barrier_algo.Interior_inner_loop(x_opt_barrier, y_opt_barrier)
print(f"f_theoretical_opt_result={problem.f(x_opt_barrier, y_original_opt_bfbm)}, f_bfbm_numerical_result={problem.f(x_opt_barrier, y_opt_barrier)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_barrier, y_original_opt_bfbm)}, g_bfbm_numerical_result={problem.g(x_opt_barrier, y_opt_barrier)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=-6.896614254074265, f_bfbm_numerical_result=-6.943511787233263
g_theoretical_opt_result=-30.894776235311067, g_bfbm_numerical_result=-30.821557852754438
h_1[0] = -0.08903923581026067
h_1[1] = -0.000533746990793324
h_1[2] = -0.9196316060015043
h_1[3] = -0.0007306402153776093
h_1[4] = -0.17196381589549836
h_1[5] = -1.0253363381458291
h_1[6] = -0.5213300781156444
h_1[7] = -0.3541883408986585
h_1[8] = -0.18790080269623421
h_1[9] = -0.0015629880718970313
h_1[10] = -1.1032786155753351
h_1[11] = -0.8808691329115683
h_1[12] = -0.0006564582975460004
h_1[13] = -0.14718743916537447
h_1[14] = -0.003772515214285399
h_1[15] = -0.9145140881356745
h_1[16] = -0.008662521590016847
h_1[17] = -0.20767764827931584
h_1[18] = -0.0010602781240690584
h_1[19] = -1.3570158760782227
h_2[0] = -0.8472008120123846
h_2[1] = -1.1456493312248355
h_2[2] = -0.9306468985573866
h_2[3] = -1.0099801699883268
h_2[4] = -0.950977715107328
h_2[5] = -1.0723219833910038
h_2[6] = -1.127054439633039
h_2[7] =

# Plot

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(times, grad_norms, marker='o', markersize=1)
plt.xlabel('CPU Time (s)')
plt.ylabel('Gradient Norm')
plt.title('Gradient Norm vs CPU Time')
plt.yscale('log')
plt.grid(True)
plt.show()
